# Курсовая работа № 2
## Аналитика. Начальный уровень

Курсовая работа состоит из двух частей – обязательной и дополнительной. **Для зачета необходимо выполнение только первой части.** Выполнение второй части может потребовать дополнительные знания Python.

- [Часть первая](#Часть-первая)
- [Часть вторая](#Часть-вторая)

## Часть первая


Перед вами стоит задача – подготовить аналитический отчет для HR-отдела. На основании проведенной аналитики предполагается составить рекомендации для отдела кадров по стратегии набора персонала, а также по взаимодействию с уже имеющимися сотрудниками.
<br><br> В базе данных лежит набор таблиц, которые содержат данные о сотрудниках вымышленной компании.
Сделайте обзор штата сотрудников компании. Составьте набор предметов исследования, а затем проверьте их на данных. Вся аналитика должна быть выполена с помощью SQL. Впоследствии данные можно визуализировать, однако финальные датафреймы для графиков также должны быть подготовлены с помощью SQL. <br><br>

Примеры гипотез:
1. Есть зависимость между `perfomance score` и тем, под чьим руководством работает сотрудник.
2. Есть зависимость между продолжительностью работы в компании и семейным положением сотрудника.
2. Есть зависимость между продолжительностью работы в компании и возрастом сотрудника.

<br><br>
Параметры для подключения следующие: хост – `dsstudents.skillbox.ru`, порт – `5432`, имя базы данных – `human_resources`, пользователь – `readonly`, пароль – `6hajV34RTQfmxhS`. Таблицы, доступные для анализа, – `hr_dataset`, `production_staff`, `recruting_costs`, `salary_grid`.

In [ ]:
import pandas as pd
import psycopg2

connection = {
    'host': 'dsstudents.skillbox.ru',
    'port': 5432,
    'dbname': 'human_resources',
    'user': 'readonly',
    'password': '6hajV34RTQfmxhS'
}
conn = psycopg2.connect(**connection)

hr_dataset_columns = pd.read_sql_query("""
    SELECT *
    FROM hr_dataset;
    """, conn).columns
hr_dataset = pd.read_sql_query("""
    SELECT *
    FROM hr_dataset;
    """, conn)
production_staff = pd.read_sql_query("""
    SELECT *
    FROM production_staff;
    """, conn)
salary_grid = pd.read_sql_query("""
    SELECT *
    FROM salary_grid;
    """, conn)
# print(hr_dataset_columns)
# print(hr_dataset)
# print(production_staff)
# print(salary_grid)

# table1 Employee Name, age, Pay Rate, state, sex, maritaldesc, citizendesc, racedesc, Days Employed, Reason For Term, 
# Employment Status, department, position, Performance Score

# table2 Employee Name, Race Desc, Reason for Term, Employment Status, Department, Position, Pay, Performance Score

# table3 Position, Hourly Mid, Salary Mid

# print(pd.read_sql_query('select distinct "citizendesc" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "Pay Rate" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "state" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "dob" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "maritaldesc" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "Hispanic/Latino" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "racedesc" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "Reason For Term" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "Employee Source" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "Performance Score" from hr_dataset', conn))
# print(pd.read_sql_query('select distinct "90-day Complaints" from production_staff', conn))

print("Есть зависимость между количеством сотрудников и штатом их происхождения")
print(pd.read_sql_query("""
    select "state", count(*) as total from hr_dataset
    group by "state"
    order by "total" desc
    """, conn))

print("Есть зависимость между расой и семейным положением")
print(pd.read_sql_query("""
    select "maritaldesc", "racedesc", count(*) as total from hr_dataset
    group by "maritaldesc", "racedesc"
    order by "racedesc"
    """, conn))

print("Есть зависимость между возрастом и Performance Score")
print(pd.read_sql_query("""
    select "age", "Performance Score", count(*) from hr_dataset
    where "Performance Score" = 'Fully Meets'
    group by "age", "Performance Score"
    order by "age"
    """, conn))

print("Есть зависимость между отделом и средней зарплатой")
print(pd.read_sql_query("""
    select s1.*, s2.* from (select distinct "position", "department" from hr_dataset) as s1
    join (select "Position", "Salary Mid" from salary_grid) as s2
    on s1."position" = s2."Position"
    order by "Salary Mid" desc
    """, conn))

Есть зависимость между количеством сотрудников и штатом их происхождения
   state  total
0     MA    275
1     CT      6
2     TX      3
3     VT      2
4     NV      1
5     KY      1
6     OH      1
7     NY      1
8     IN      1
9     FL      1
10    ME      1
11    NC      1
12    RI      1
13    GA      1
14    ID      1
15    PA      1
16    MT      1
17    TN      1
18    WA      1
19    AL      1
20    UT      1
21    CO      1
22    VA      1
23    CA      1
24    AZ      1
25    NH      1
26    OR      1
27    ND      1
Есть зависимость между расой и семейным положением
   maritaldesc                          racedesc  total
0    Separated  American Indian or Alaska Native      1
1     Divorced  American Indian or Alaska Native      2
2      Married  American Indian or Alaska Native      1
3     Divorced                             Asian      3
4       Single                             Asian     13
5    Separated                             Asian      1
6      Widowed      

## Часть вторая

Перед вами стоит задача – подготовить аналитический ответ для SMM-отдела компании Skillbox. <br> Объектом анализа является  [паблик Skillbox Вконтакте](https://vk.com/skillbox_education). <br> <br> 
Подключитесь к  API VK и выгрузите посты со стены паблика Skillbox за интересующий период (определите самостоятельно и обоснуйте). Проанализируйте влияние различных факторов (например, времени публикации) на вовлеченность пользователей (количество лайков, комментариев, голосов в опросах). Сделайте аналитику по рубрикам (примеры рубрик: дизайн-битва, игра по управлению), которые есть в паблике. Выбрать нужные посты можно с помощью регулярных выражений. Составьте перечень рекомандаций для SMM-отдела по итогам анализа. <br> <br> 

Дополнительные инструкции по работе с API VK расположены [здесь](https://nbviewer.jupyter.org/github/vikaborel/dstutorials/blob/master/Дополнительные%20инструкции.ipynb).

In [ ]:
### YOUR CODE HERE ###


